In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('../')
import pandas as pd 
import numpy as np 
import datetime
from src.utils.io.read import RawDataReader
from src.settings import Settings
from src.utils.dates import get_nyse_business_dates

In [2]:
start_date = datetime.date(2010,1,1)
end_date =  datetime.date(2025,7,15)

In [3]:
business_days = get_nyse_business_dates(start_date, end_date)

In [4]:
business_days

[datetime.date(2010, 1, 4),
 datetime.date(2010, 1, 5),
 datetime.date(2010, 1, 6),
 datetime.date(2010, 1, 7),
 datetime.date(2010, 1, 8),
 datetime.date(2010, 1, 11),
 datetime.date(2010, 1, 12),
 datetime.date(2010, 1, 13),
 datetime.date(2010, 1, 14),
 datetime.date(2010, 1, 15),
 datetime.date(2010, 1, 19),
 datetime.date(2010, 1, 20),
 datetime.date(2010, 1, 21),
 datetime.date(2010, 1, 22),
 datetime.date(2010, 1, 25),
 datetime.date(2010, 1, 26),
 datetime.date(2010, 1, 27),
 datetime.date(2010, 1, 28),
 datetime.date(2010, 1, 29),
 datetime.date(2010, 2, 1),
 datetime.date(2010, 2, 2),
 datetime.date(2010, 2, 3),
 datetime.date(2010, 2, 4),
 datetime.date(2010, 2, 5),
 datetime.date(2010, 2, 8),
 datetime.date(2010, 2, 9),
 datetime.date(2010, 2, 10),
 datetime.date(2010, 2, 11),
 datetime.date(2010, 2, 12),
 datetime.date(2010, 2, 16),
 datetime.date(2010, 2, 17),
 datetime.date(2010, 2, 18),
 datetime.date(2010, 2, 19),
 datetime.date(2010, 2, 22),
 datetime.date(2010, 2, 23

In [8]:
import pandas as pd
from datetime import datetime, date
from typing import List, Union, Optional
import calendar

 
def get__roll_dates(start_date: Union[str, date, datetime], 
                       end_date: Union[str, date, datetime],
                       roll_start_day: int = 5,
                       roll_end_day: int = 9,
                       business_days: List[date] = None) -> pd.DataFrame:
    """
    Returns a DataFrame with roll dates for each month between start_date and end_date.
    For example, BCOM (Bloomberg Commodity Index) typically rolls between the 5th and 9th BUSINESS DAYS.
    
    Parameters:
    -----------
    start_date : str, date, or datetime
        Start date of the range
    end_date : str, date, or datetime  
        End date of the range
    roll_start_day : int, default 5
        Which business day of the month when rolling period starts (e.g., 5 = 5th business day)
    roll_end_day : int, default 9
        Which business day of the month when rolling period ends (e.g., 9 = 9th business day)
    business_days : List[date], optional
        List of business days to use. If None, uses pandas default (excludes weekends)
        
    Returns:
    --------
    pd.DataFrame : DataFrame with columns:
        - month: YYYY-MM format
        - current_roll_start_date: Nth business day for roll start (e.g., 5th business day)
        - current_roll_end_date: Nth business day for roll end (e.g., 9th business day)
        - previous_roll_start_date: Previous month's Nth business day for roll start
        - previous_roll_end_date: Previous month's Nth business day for roll end
        - next_roll_start_date: Next month's Nth business day for roll start
        - next_roll_end_date: Next month's Nth business day for roll end
    """
    
    # Convert inputs to datetime if they're strings
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date).date()
    elif isinstance(start_date, datetime):
        start_date = start_date.date()
        
    if isinstance(end_date, str):
        end_date = pd.to_datetime(end_date).date()
    elif isinstance(end_date, datetime):
        end_date = end_date.date()
    
    # Generate all roll dates for extended range (include previous/next months)
    # Extend range by 1 month before and 3 months after to ensure we have complete data
    extended_start = pd.Timestamp(start_date.replace(day=1)) - pd.DateOffset(months=1)
    extended_end = pd.Timestamp(end_date.replace(day=1)) + pd.DateOffset(months=3)
    
    def get_nth_business_day_of_month(year: int, month: int, n: int) -> Optional[date]:
        """Get the nth business day of a given month"""
        # Get first and last day of month
        month_start = date(year, month, 1)
        last_day = calendar.monthrange(year, month)[1]
        month_end = date(year, month, last_day)
        
        if business_days is not None:
            # Use provided business days
            month_business_days = [d for d in business_days 
                                 if month_start <= d <= month_end]
        else:
            # Use pandas business days (excludes weekends)
            month_business_days = pd.bdate_range(start=month_start, end=month_end)
            month_business_days = [d.date() for d in month_business_days]
        
        # Return the nth business day (1-indexed)
        if len(month_business_days) >= n:
            return month_business_days[n - 1]  # Convert to 0-indexed
        else:
            return None  # Month doesn't have enough business days
    
    # Generate monthly range for extended period
    monthly_range_extended = pd.date_range(start=extended_start, end=extended_end, freq='MS')
    
    # Generate all roll dates in extended range
    all_roll_dates = {}  # {year_month: {'start': date, 'end': date}}
    
    for month_start in monthly_range_extended:
        year_month = month_start.strftime('%Y-%m')
        year = month_start.year
        month = month_start.month
        
        # Get nth business day for roll start
        roll_start = get_nth_business_day_of_month(year, month, roll_start_day)
        
        # Get nth business day for roll end  
        roll_end = get_nth_business_day_of_month(year, month, roll_end_day)
            
        all_roll_dates[year_month] = {
            'start': roll_start,
            'end': roll_end
        }
    
    # Create DataFrame for the requested date range only
    monthly_range_requested = pd.date_range(start=start_date.replace(day=1), end=end_date.replace(day=1), freq='MS')
    
    results = []
    for month_start in monthly_range_requested:
        year_month = month_start.strftime('%Y-%m')
        
        # Get previous month
        prev_month = month_start - pd.DateOffset(months=1)
        prev_key = prev_month.strftime('%Y-%m')
        
        # Get next month
        next_month = month_start + pd.DateOffset(months=1)
        next_key = next_month.strftime('%Y-%m')
        
        # Build row
        row = {
            'month': year_month,
            'current_roll_start_date': all_roll_dates.get(year_month, {}).get('start'),
            'current_roll_end_date': all_roll_dates.get(year_month, {}).get('end'),
            'previous_roll_start_date': all_roll_dates.get(prev_key, {}).get('start'),
            'previous_roll_end_date': all_roll_dates.get(prev_key, {}).get('end'),
            'next_roll_start_date': all_roll_dates.get(next_key, {}).get('start'),
            'next_roll_end_date': all_roll_dates.get(next_key, {}).get('end')
        }
        
        results.append(row)
    
    return pd.DataFrame(results)


In [9]:
start_date,end_date

(datetime.date(2010, 1, 1), datetime.date(2025, 7, 15))

In [10]:
results = get_bcom_roll_dates(start_date = start_date, 
                                                       end_date = end_date,
                                                       roll_start_day  = 5,
                                                       roll_end_day = 9,
                                                       business_days = business_days) 

In [12]:
results.tail(20)

,month,current_roll_start_date,current_roll_end_date,previous_roll_start_date,previous_roll_end_date,next_roll_start_date,next_roll_end_date
167,2023-12,2023-12-07,2023-12-13,2023-11-07,2023-11-13,2024-01-08,2024-01-12
168,2024-01,2024-01-08,2024-01-12,2023-12-07,2023-12-13,2024-02-07,2024-02-13
169,2024-02,2024-02-07,2024-02-13,2024-01-08,2024-01-12,2024-03-07,2024-03-13
170,2024-03,2024-03-07,2024-03-13,2024-02-07,2024-02-13,2024-04-05,2024-04-11
171,2024-04,2024-04-05,2024-04-11,2024-03-07,2024-03-13,2024-05-07,2024-05-13
172,2024-05,2024-05-07,2024-05-13,2024-04-05,2024-04-11,2024-06-07,2024-06-13
173,2024-06,2024-06-07,2024-06-13,2024-05-07,2024-05-13,2024-07-08,2024-07-12
174,2024-07,2024-07-08,2024-07-12,2024-06-07,2024-06-13,2024-08-07,2024-08-13
175,2024-08,2024-08-07,2024-08-13,2024-07-08,2024-07-12,2024-09-09,2024-09-13
176,2024-09,2024-09-09,2024-09-13,2024-08-07,2024-08-13,2024-10-07,2024-10-11
